# Mapping the CT PPP data

Import necessary libraries. To avoid truncating the number of rows displayed in output, the max_rows below can be set to 'None'. Because output can be over 60K rows, this limit is useful to start with.

In [ ]:
import numpy as np
import pandas as pd
import json
from matplotlib import pyplot as plt
import geopandas as gpd
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:,.0f}".format

## Loading and preparing the data

Read in the three JSON files that contain the CT PPP data. Unless the JSON files are in the same working directory as this notebook when working locally on your computer, it will necessary to use full path names for the files. <br><br>If using Google Colab, first upload these JSON files to Colab by clicking the folder icon at the notebook's sidebar and then clicking the upload button. Then preface the JSON filenames below with '/content/', so that the first filename below is '/content/ctppp_small_063020.json' instead. <br><br>Here we assign <\\$150K loans to the small_df dataframe, the >\\$150K loans to the large_df dataframe, and the composite dataset to the total_df dataframe.

In [ ]:
small_df = pd.read_json('ctppp_small_063020.json', dtype={'Zip': 'str'})

In [ ]:
large_df = pd.read_json('ctppp_large_063020.json', dtype={'Zip': 'str'})

In [ ]:
total_df = pd.read_json('ctppp_total_063020.json', dtype={'Zip': 'str'})

One simple approach for creating point estimates of the >\\$150K loans is to assign the same dollar value for all loans in the same dollar range. E.g., \\$7.5M for all \\$5M-10M loans. This is not an actual estimate per se, but a standardization of the larger loan values. For the sake of simplicity, this approach is used below to illustrate aggregrate loan dollar amounts. This standardization uses the following steps:
1. Use the reported SBA total loan dollar amount for CT and subtract out deleted loans from this SBA total. In subtracting out the deleted loans, for <\\$150K loans, use the actual loan amount reported; for >\\$150K loans, use the midpoint of the dollar range for that loan. E.g., \\$250K for \\$150K-\\$350K loans.
2. Compute a percentage of the dollar range that will be added to the loan mininum to give the standardized loan value. Use the same percentage for all loan ranges. E.g., 50% would yield the midpoint for all the loan ranges. In the initial tranche of data provided by the SBA, 35.425% was deemed appropriate after cleaning the data and subtracting out deleted items.
3. Follow this approach to create another standardized column of loan amounts in the large and total loan dataframes. Actual reported loan amounts for the <\\$150K loans are still always used in this new column, however.

In [ ]:
standardization_percentage = 0.35425
large_df['StandardLoanAmount'] = (large_df['LoanMin']+(large_df['LoanMax']-large_df['LoanMin'])*standardization_percentage)
total_df['StandardLoanAmount'] = (total_df['LoanMin']+(total_df['LoanMax']-total_df['LoanMin'])*standardization_percentage)

With these standardized loan amounts, it is easier to illustrate aggregate relationships in the data. The CT PPP data is now fully available in the notebook for analysis.

## Mapping the data

The Python library used here for mapping is Geopandas. First load Connecticut's zip code information from the GEOJSON file. Then merge this geographical information with data from the one of the existing dataframes to produce state maps that illustrate the data spatially over zip codes. The GEOJSON file is assumed to be in the working directory.

In [ ]:
fp = 'ct_zip_code_boundaries.geojson'
map_df = gpd.read_file(fp)
map_df.head()

In [ ]:
map_df.rename(columns={'zcta5ce10': 'Zip'}, inplace=True)

In [ ]:
map_df.info()

Before creating the map, prepare another dataframe used to put the major cities of Connecticut onto the state map. The JSON file containing these coordinates is assumed to be in the working directory.

In [ ]:
cities_df = pd.read_json('cities_for_map.json',dtype={'latitude': 'float','longitude': 'float'})
cities_df['coordinates'] = list(zip(cities_df.longitude, cities_df.latitude))

from shapely.geometry import Point
cities_df['coordinates'] = cities_df['coordinates'].apply(Point)
cities_map_df = gpd.GeoDataFrame(cities_df, geometry='coordinates')
cities_map_df.crs= 'EPSG:4326'

Plot the map, filling the zip codes that have null values with zero to ensure their zip code polygons are plotted. Find the largest number of jobs in a zip code in order to manually label the color bar. The color bar coordinates have also been determined manually.

In [ ]:
total_df.groupby('Zip')['JobsRetained'].sum().max()

In [ ]:
total_jobs_amount_by_zip = total_df.groupby('Zip')['JobsRetained'].sum()
map_total_jobs_amount_df = pd.merge(map_df, total_jobs_amount_by_zip, on='Zip', how='left')
map_total_jobs_amount_df['JobsRetained'].fillna(0, inplace=True)

plt.rcParams['figure.figsize'] = (11.15,7.75)
fig, ax = plt.subplots(1,1)
ax.axis('off')

ax.set_title(label='Total retained jobs across CT from PPP lending',
                     fontdict={'fontsize': 18,'fontweight': 'bold'})

vmin = 0
vmax = 13
cax = fig.add_axes([0.60, 0.22, 0.25, 0.015])

cb = fig.colorbar(label='jobs by zip code (000s)', shrink = 0.25, ax=ax, cax=cax, orientation='horizontal', 
             mappable=plt.cm.ScalarMappable(cmap='Blues',norm=plt.Normalize(vmin=vmin,vmax=vmax)))
map_total_jobs_amount_df.plot(column='JobsRetained', cmap='Blues', edgecolor='gray',
                 linewidth=0.25, ax=ax)

cities_map_df.plot(ax=ax, color='black', alpha = 0.5)

for x, y, label in zip(cities_map_df.coordinates.x, cities_map_df.coordinates.y, cities_map_df.city):
    if label in ['Stamford', 'Bridgeport', 'New Haven', 'Hartford', 'Waterbury']:
        ax.annotate(label, xy=(x,y), xytext=(4,4), textcoords='offset points', size=12)
    else:
        ax.annotate(label, xy=(x,y), xytext=(4,4), textcoords='offset points', size=10)
        
plt.savefig('map_total_jobs_by_zip.png', bbox_inches='tight', dpi=600, height=7.75, width=11.15, units='in', transparent=True)